In [1]:
#import necessary libraries 

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [5]:
#we will laod our datasets 

email_data = pd.read_csv('spam_data/email-spam/email.csv', encoding='latin1')
sms_data = pd.read_csv('spam_data/sms-spam/spam.csv', encoding='latin1') 
combo_data = pd.read_csv('spam_data/hybrid-spam/rcs_sms_spam.csv', encoding='latin1') 


email_df = pd.DataFrame(email_data)
sms_df = pd.DataFrame(sms_data)
combo_df = pd.DataFrame(combo_data) 

In [17]:
#in order to do an Isolation Forest on this data we will need to do some processing to get into the right form 
#fix xombo bugs
combo_df.columns = combo_df.columns.str.strip()
combo_df = combo_df.rename(columns={'ï»¿Message':'Message'})
valid_labels = ['ham', 'spam']
combo_df = combo_df[combo_df['Category'].isin(valid_labels)]
email_df = email_df[email_df['Category'].isin(valid_labels)]


sms_spam_df = sms_df[sms_df['v1']=='spam'] #our "mal" data frame 
sms_ham_df = sms_df[sms_df['v1']=='ham'] # our "normal" data frame 

email_spam_df = email_df[email_df['Category'] == 'spam']
email_ham_df = email_df[email_df['Category'] == 'ham']

combo_spam_df = combo_df[combo_df['Category'] == 'spam']
combo_ham_df = combo_df[combo_df['Category'] == 'ham']


#now that the datasets are split up we can get our data isolated (remove labels) 
email_ham_df = email_ham_df.loc[:, 'Message'] 
sms_ham_df = sms_ham_df.loc[:, 'v2'] 

sms_ham_df.head(3)



0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
3    U dun say so early hor... U c already then say...
Name: v2, dtype: object

In [18]:
#now we can proceed to create the isolation forests 
from scipy.sparse import hstack, csr_matrix, vstack #MOVE THIS INTO TOP BOX WHEN JUPYTER STOPS BUGGING OUT

vectorizer1 = TfidfVectorizer(stop_words='english')
vectorizer2 = TfidfVectorizer(stop_words='english')

sms_ham_vect = vectorizer1.fit_transform(sms_ham_df)
email_ham_vect = vectorizer2.fit_transform(email_ham_df)

sms_spam_vect = vectorizer1.transform(sms_spam_df)
email_spam_vect = vectorizer2.transform(email_spam_df)

vectorizer_combo = TfidfVectorizer(stop_words='english')
combo_ham_vect_msg = vectorizer_combo.fit_transform(combo_ham_df['Message'])
combo_spam_vect_msg = vectorizer_combo.transform(combo_spam_df['Message'])


combo_other_ham = combo_ham_df[['MessageType', 'Verified']].values
combo_other_spam = combo_spam_df[['MessageType', 'Verified']].values
combo_ham_df = combo_ham_df.loc[:, 'Message'] 

#print("len of combo_other_train", len(combo_other_train), "len of combo_mssage_train", len(combo_msg_train_vect))
combo_data_ham = hstack([combo_ham_vect_msg, csr_matrix(combo_other_ham)], format='csr')
combo_data_spam = hstack([combo_spam_vect_msg, csr_matrix(combo_other_spam)], format='csr')


# train isolation forest on only legit messages
sms_iso = IsolationForest(contamination='auto', random_state=42)
sms_iso.fit(sms_ham_vect)

email_iso = IsolationForest(contamination='auto', random_state=42)
email_iso.fit(email_ham_vect)

combo_iso = IsolationForest(contamination='auto', random_state=42)
combo_iso.fit(combo_data_ham)


,n_estimators,100
,max_samples,'auto'
,contamination,'auto'
,max_features,1.0
,bootstrap,False
,n_jobs,None
,random_state,42
,verbose,0
,warm_start,False


In [22]:
# now we can predict based on the original values as well as the spam values 

sms_ham_scores = sms_iso.predict(sms_ham_vect)
sms_spam_scores = sms_iso.predict(sms_spam_vect)

combo_ham_scores = combo_iso.predict(combo_data_ham)
combo_spam_scores = combo_iso.predict(combo_data_spam)


email_ham_scores = email_iso.predict(email_ham_vect)
email_spam_scores = email_iso.predict(email_spam_vect)

print("SMS HAM Average: ", np.average(sms_ham_scores))
print("SMS SPAM Average: ", np.average(sms_spam_scores))

print("Email HAM Average: ", np.average(email_ham_scores))
print("Email SPAM Average: ", np.average(email_spam_scores))


#ACTUALLY RETURNS DIFFERENT RESULTS? UNEXPECTED.
print("Combo HAM Average: ", np.average(combo_ham_scores))
print("Combo SPAM Average: ", np.average(combo_spam_scores))

#can get metrics by converting to 0 fir ham and 1 for spam
combo_data = vstack([combo_data_ham, combo_data_spam])
combo_labels = ([0] * combo_data_ham.shape[0] + [1] * combo_data_spam.shape[0])

combo_preds = combo_iso.predict(X_combo_test)

combo_preds_numbies = (combo_preds == -1).astype(int)


print("Combo Isolation Forest")
print("Accuracy:", accuracy_score(combo_labels, combo_preds_binary))
print("Precision:", precision_score(combo_labels, combo_preds_numbies, zero_division=0))
print("Recall:", recall_score(combo_labels, combo_preds_numbies, zero_division=0))
print("F1:", f1_score(combo_labels, combo_preds_numbies, zero_division=0))





SMS HAM Average:  1.0
SMS SPAM Average:  1.0
Email HAM Average:  1.0
Email SPAM Average:  1.0
Combo HAM Average:  0.8461538461538461
Combo SPAM Average:  1.0
Combo Isolation Forest
Accuracy: 0.631578947368421
Precision: 0.0
Recall: 0.0
F1: 0.0


In [26]:
#clearly based on the aveeraeges the iso forest does not detect any spam for email or messages 
#but we can still get some stats for comparison easily because of this 

#for sms
tp = np.sum(sms_ham_scores==1) #model guesses ham as ham correctly always 
fp = np.sum(sms_spam_scores==1) # model guesses spam as ham incorrectly always...,
fn = tn = 0 

#!!!!!!!
#NOTE FOR FUTURE: THESE RESULTS ONLY WORK BECAUSE OF THE MESSED UP ISO FOREST !!!!1
#!!!!!!!!
print("SMS ISO FOREST RESULTS") 
print("Precision: ",tp / (tp + fp) if (tp + fp) > 0 else 0 ) 
print("Recall: ",tp / (tp + fn) if (tp + fn) > 0 else 0 ) 
print("Accuracy: ",tp/(fp+tp) ) 


#for email 
tp = np.sum(email_ham_scores==1) #model guesses ham as ham correctly always 
fp = np.sum(email_spam_scores==1) # model guesses spam as ham incorrectly always...,
fn = tn = 0 

print("\n\nEmail ISO FOREST RESULTS") 
print("Precision: ",tp / (tp + fp) if (tp + fp) > 0 else 0 ) 
print("Recall: ",tp / (tp + fn) if (tp + fn) > 0 else 0 ) 
print("Accuracy: ",tp/(fp+tp) ) 

SMS ISO FOREST RESULTS
Precision:  0.9989648033126294
Recall:  1.0
Accuracy:  0.9989648033126294


Email ISO FOREST RESULTS
Precision:  0.9995856639734825
Recall:  1.0
Accuracy:  0.9995856639734825


In [36]:

print("\nCombo Random Forest Regression + Logic with Custom Data ")
hits = misses = 0 
for index,row in combo_df.iterrows(): 
    if row['Category'] == 'spam': 
        if predict_spam(row) == 1: 
            hits+=1 
            continue 
        else: 
            misses += 1 
            continue 
    if row['Category'] == 'ham': 
        if predict_spam(row) == -1:
            hits += 1 
            continue 
        else: 
            misses+=1
            continue

print("Accuracy: ", hits / (hits + misses))

print("\nSMS Random Forest with Custom Data ")

hits = misses = 0 
for index,row in combo_df.iterrows(): 
    message_vect = vectorizer1.transform([row['Message']])
    prediction = sms_iso.predict(message_vect) 

    if prediction == 1:
        if row['Category'] == 1:
            hits+=1 
        else:
            misses +=1 
    else:
        if row['Category'] == -1:
            hits +=1 
        else:
            misses += 1

print("Accuracy: ", hits / (hits + misses))

    


Combo Random Forest Regression + Logic with Custom Data 
verified ham
verified ham
verified ham
verified ham
verified ham
Accuracy:  0.5789473684210527

SMS Random Forest with Custom Data 
Accuracy:  0.0


In [35]:
#Now we will implement our hybrid model that uses rcs and sms as paramers in addition to the message 
def predict_spam(data):
   # print("Message: ", data['Message'])
    if data['MessageType'] == 1: #RCS 
        if data['Verified'] ==1: #verified by Google 
            print("verified ham")
            return -1 # not spam 
             #SMS or unverified RCS 
    msg_vect = vectorizer1.transform([data['Message']])#sti.l need to convert message
    #print("USING ML", sms_forest.predict(msg_vect)[0])
    return sms_iso.predict(msg_vect)[0]

